In [ ]:
pip install torch transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

PRETRAINED_LM = 'selimyagci/bert-misogyny-english' # if the model already in HuggingFace hub
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_LM) # or like 'google-bert/bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_LM)
# if you are loading the model locally comment above line and use the below
# model = torch.load(PATH, map_location=torch.device('cpu'))

In [ ]:
def inference(text):
  input = tokenizer(text, return_tensors='pt')
  logits = model(**input).logits
  predicted_class_id = logits.argmax().item()
  from torch.nn.functional import softmax
  probability = (softmax(logits)).data[0][1].item()
  s = round(probability,2)
  return s # or you can return predicted_class_id, whereas probability returns a value between 0 and 1

In [ ]:
print(inference('put your text to detect here'))

In [7]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report

def encode(docs):
  encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length', return_attention_mask=True, truncation=True, return_tensors='pt')
  input_ids = encoded_dict['input_ids']
  attention_masks = encoded_dict['attention_mask']
  return input_ids, attention_masks

def evaluateOnTest(PATH):
  test = pd.read_csv(PATH)

  test_input_ids, test_att_masks = encode(test['text'].values.tolist())
  test_y = torch.LongTensor(test['label'].values.tolist())

  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  model.eval()
  test_pred = []
  test_loss= 0
  with torch.no_grad():
      for step_num, batch_data in tqdm(enumerate(test_dataloader)):
          input_ids, att_mask, labels = [data.to(device) for data in batch_data]
          output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

          loss = output.loss
          test_loss += loss.item()

          test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
  test_pred = np.concatenate(test_pred)

  return classification_report(test_pred, test['label'].to_numpy(),target_names=['nothate','hate'])

In [ ]:
print(evaluateOnTest('putYourTestDataPath')) # run this to get the evaluation report for your test data